# Feature documentation

## Features Actuales

   ###  MaxMin Date: 
 - Saco la date maxima y minima por usuario y de ahi obtengo features de esas dates
    
## Features probados que no funcionaron y otras cosas que tampoco:

   ### Cantidad de eventos por tipo
  - Numero de veces que realizo cada evento en dataframe
   ### Algoritmo 0.63 con RandomFores
  - Use random forest para ver la importancia de lo features de las busquedas y de los modelos y me quede con los mejores 10, el resultado fue menor con 0.61
   
## Best Score: 0.71 
  - commit 0.71 
  - Max, min date.
               
## Cosas a probar:
  - features 
  - model countvector (sin random forest seleccionando un top)
  - most viewed and searched
  - lead and conversion
  - most common search (sin random forest seleccionando un top)
  - diff caro
 
 
 
## Acotaciones
     

In [20]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from scipy.sparse import hstack
from xgboost.sklearn import XGBClassifier # <3
from sklearn.model_selection import train_test_split
import gc
import matplotlib.pyplot as plt
from scipy.sparse.csr import csr_matrix
from sklearn.model_selection import train_test_split
import featuretools as ft

pd.options.mode.chained_assignment = None


In [22]:
df = pd.read_csv('events_up_to_01062018.csv', low_memory = False)
df_labels= pd.read_csv('labels_training_set.csv', low_memory = False)


In [23]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values(['timestamp','event'])
df = df.reset_index(drop=True)

In [24]:
df_last_date = df.groupby('person').agg({'timestamp':['max', 'min']}).reset_index()
df_last_date.head()

person           timestamp                    
                            max                 min
0  0008ed71 2018-05-17 16:28:37 2018-05-17 12:27:47
1  00091926 2018-05-31 19:52:03 2018-05-03 22:08:29
2  00091a7a 2018-03-26 14:56:58 2018-03-26 14:51:11
3  000ba417 2018-05-26 13:09:22 2018-05-17 11:11:45
4  000c79fe 2018-05-29 00:38:07 2018-05-29 00:27:47

Consigo features para cada persona, en este caso son la primer y ultima fecha que se metio en estos datos.

In [25]:
df_last_date.columns = ['person','max', 'min']

In [26]:
es = ft.EntitySet(id = 'person')
es = es.entity_from_dataframe(entity_id = 'person_id', dataframe = df_last_date, index = 'person')

In [27]:
%time features, feature_names = ft.dfs(entityset=es, target_entity='person_id', max_depth = 2)

CPU times: user 1.82 s, sys: 0 ns, total: 1.82 s
Wall time: 1.83 s


Aca utilizo la libreria 'featuretools' y me hace una cantidad de features con las fechas

In [28]:
features['person'] = features.index
features = features.reset_index(drop=True)
features.head()

,DAY(max),DAY(min),YEAR(max),YEAR(min),MONTH(max),MONTH(min),WEEKDAY(max),WEEKDAY(min),person
0,17,17,2018,2018,5,5,3,3,0008ed71
1,31,3,2018,2018,5,5,3,3,00091926
2,26,26,2018,2018,3,3,0,0,00091a7a
3,26,17,2018,2018,5,5,5,3,000ba417
4,29,29,2018,2018,5,5,1,1,000c79fe


In [29]:
features = features.drop('YEAR(max)',axis = 1)
features = features.drop('YEAR(min)',axis = 1)

Saco el año ya que los datos estan todos en el mismo año

In [30]:
features.to_csv(path_or_buf = 'last_date.csv', index = False)

(no es necesario lo hice para pasar los features rapido a otros notebooks sin repetir mi codigo ahi)

## XGboost entrenamiento

In [31]:
df_train = df_labels.merge(features , left_on='person', right_on='person' , how='left')

Los labels me dan mi set para entrenar, los que no se encuentran en labels tengo que predecirlos

In [32]:
df_train.head()

,person,label,DAY(max),DAY(min),MONTH(max),MONTH(min),WEEKDAY(max),WEEKDAY(min)
0,0566e9c1,0,31,22,5,5,3,1
1,6ec7ee77,0,28,28,5,5,0,0
2,abe7a2fb,0,29,10,5,2,1,5
3,34728364,0,27,18,5,5,6,4
4,87ed62de,0,19,18,5,5,5,4


Si ven aca, de la columna label en adelante tenemos los features.

In [41]:
X, y = df_train.iloc[:,2:],df_train.iloc[:,1]
X = X.fillna(0)
X.head()

,DAY(max),DAY(min),MONTH(max),MONTH(min),WEEKDAY(max),WEEKDAY(min)
0,31,22,5,5,3,1
1,28,28,5,5,0,0
2,29,10,5,2,1,5
3,27,18,5,5,6,4
4,19,18,5,5,5,4


Separamos los datos para hacer xgboost de la siguiente forma


|Variable |Contiene|
|------------------------|-----------------------------------------------------|
|X| features que usa xgboost son solo numeros es decir que sacamos a la persona   |
|y| label de cada persona|

## Xgboost

Para evaluar usen esta medida que me da valores muy parecidos a los de kaggle, para hacer las predicciones usen el otro

In [42]:
my_classifier1 = xgb.XGBClassifier(objective ='reg:linear', 
                colsample_bytree = 1, learning_rate = 0.1,
                max_depth = 6,
                subsample = 0.8,
                gamma = 1,
                n_estimators = 10)

Este es el arbol con sus hiperparametros

In [43]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

Aca vuelvo a separar los datos para poder realizar una metrica interna y ver masomenos como performan las cosas que hago

In [49]:
persons = df_labels['person']
persons_to_predict = (features.loc[~features['person'].isin(persons)])['person'].to_frame()
persons_to_predict.head()

,person
1,00091926
2,00091a7a
3,000ba417
5,000e4d9e
6,000e619d


Obtengo las personas a predecir!

In [59]:
X_predict = features.loc[features['person'].isin(persons_to_predict['person'])]
X_predict = X_predict.drop(['person'], axis=1)
X_predict.head()

,DAY(max),DAY(min),MONTH(max),MONTH(min),WEEKDAY(max),WEEKDAY(min)
1,31,3,5,5,3,3
2,26,26,3,3,0,0
3,26,17,5,5,5,3
5,30,16,5,5,2,2
6,18,15,5,5,4,1


Necesito tener el mismo dataframe que tenia cuando lo entrene pero ahora para predecir, en este caso <b>X</b>

In [52]:
my_classifier1.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=1, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

Entreno al arbol!

In [53]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,  my_classifier1.predict_proba(X_test)[:,1])

0.7343839517167443

Obtengo un resultado con los que separe para el test mas arriba en :
```python
    X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)
    ```
###### Lo que hace es sacar las predicciones para X_test y evaluarlos con y_test
    

In [60]:
entrie = my_classifier1.predict_proba(X_predict)[:,1]

Valores a subir a kaggle, lo que sigue son transformaciones a dataframe para convertirlo en csv, y algunas cosas para darme idea tipo por donde andan los valores normales en los resultados que me da

In [61]:
entrie

array([0.22109339, 0.26868096, 0.19177619, ..., 0.18967366, 0.21788752,
       0.18967366], dtype=float32)

In [62]:
seriesita = pd.Series(entrie)

In [65]:
df_entrie = persons_to_predict
df_entrie['label'] = seriesita

In [66]:
df_entrie.head()

,person,label
1,00091926,0.268681
2,00091a7a,0.191776
3,000ba417,0.208852
5,000e4d9e,0.189674
6,000e619d,0.208852


In [67]:
df_entrie = df_entrie.fillna(0)

In [68]:
num = df_entrie._get_numeric_data()
num[num < 0] = 0

In [69]:
df_entrie.to_csv(path_or_buf = 'submit_kaggle.csv', index = False)

In [70]:
df_entrie.shape

(19415, 2)

In [71]:
df_entrie['label'].nlargest()

15871    0.443819
13834    0.435719
459      0.410319
10498    0.400190
5378     0.395132
Name: label, dtype: float32

 # Random Forest feature importance
   - Algoritmo usado para sacar importancia de los features y ver cuales no nos estan sirviendo al modelo
 #### Es parecido a lo que hice en xgboost con algunas cosas magicas, que use para ponerlo en df y ver resultados
 
  ## NOTAR :
   - Que use la X de xgboost
   - Y que el codigo abajo del dataframe comentado puede funcionar para separar los df pidiendo que nos deje las columnas esas

In [77]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
#Load boston housing dataset as an example

names = X.columns
rf = RandomForestRegressor()
rf.fit(X, y)
zipped = zip(map(lambda x: round(x, 4), rf.feature_importances_), names)
feature = sorted(zipped, key=lambda x: x[1])

/home/amaherok/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [78]:
feat_importance = pd.DataFrame(feature, columns=['importance', 'feature'])
feat_importance.sort_values('importance', ascending=False).head(100)

,importance,feature
1,0.2704,DAY(min)
0,0.2576,DAY(max)
5,0.1511,WEEKDAY(min)
3,0.1419,MONTH(min)
4,0.1397,WEEKDAY(max)
2,0.0392,MONTH(max)


In [80]:
#feat_importance =feat_importance.loc[feat_importance['importance'].isin(feat_importance['importance'].nlargest(2))]
#feats_servibles = feat_importance['feature'].tolist()
#feats_servibles
#features = features[feats_servibles]

['DAY(max)', 'DAY(min)']